In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import random
import os

In [ ]:
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-c6bf4914-9520-3a69-b4d0-9d9e2f9b071a)


In [ ]:
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

#Get helper functions
from helper_functions import unzip_data, create_tensorboard_callback, plot_loss_curves, compare_historys

--2022-05-09 01:45:59--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2022-05-09 01:46:00 (100 MB/s) - ‘helper_functions.py’ saved [10246/10246]



In [ ]:
!wget https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip

#Unzip dataset
unzip_data('nlp_getting_started.zip')

--2022-05-09 01:46:00--  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.199.128, 74.125.20.128, 108.177.98.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.199.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K) [application/zip]
Saving to: ‘nlp_getting_started.zip’

nlp_getting_started 100%[===================>] 593.11K  --.-KB/s    in 0.004s  

2022-05-09 01:46:00 (151 MB/s) - ‘nlp_getting_started.zip’ saved [607343/607343]



In [ ]:
#For us to import data using pandas, well need to fit all the data on the ram, which fails when the data size is huge
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")
train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [ ]:
#Shuffle dataframe
train_df_shuffled = train_df.sample(frac=1, random_state=42)
train_df_shuffled.head()

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0


In [ ]:
#Lets visualize some random training samples
random_index= random.randint(0, len(train_df_shuffled)-5)
for row in train_df_shuffled[['text', 'target']][random_index:random_index+5].itertuples():
  _, text, target = row
  print (f"Target : {target}","(real disaster)" if target > 0 else "(not real disaster)")
  print (f"Text : \n{text}\n")
  print ("------\n")

Target : 1 (real disaster)
Text : 
Reminder: Mass murderer and white supremacist Anders Breivik was also unsurprisingly an anti-feminist.
http://t.co/1lXnJVl8TR

------

Target : 0 (not real disaster)
Text : 
do he love me do he love me not I ain't a playa I just crush a lot

------

Target : 1 (real disaster)
Text : 
Julian Knight - @SCVSupremeCourt dismisses mass murderer's attempt to increase prisoner pay. Challenged quantum of 5% increase 2013.

------

Target : 1 (real disaster)
Text : 
Savings and sewing in Guatemala: Savings and sewing in Guatemala. When a natural disaster hit seamstress Elvia...  http://t.co/jdx9OX2kIk

------

Target : 0 (not real disaster)
Text : 
Man Cam just demolished his plate. His ass was hungry

------



In [ ]:
from sklearn.model_selection import train_test_split

train_sentences, val_sentences, train_labels, val_labels = train_test_split(train_df_shuffled['text'].to_numpy(),
                                                                            train_df_shuffled['target'].to_numpy(),
                                                                            test_size=0.1,
                                                                            random_state=42)

In [ ]:
len(train_sentences), len(train_labels), len(val_sentences), len(val_labels)

(6851, 6851, 762, 762)

In [ ]:
from tensorflow.keras.layers import TextVectorization

#Using default Parameters
text_vectorizer = TextVectorization(max_tokens=None,
                                    standardize='lower_and_strip_punctuation',
                                    split='whitespace',
                                    ngrams=None,
                                    output_mode='int',
                                    output_sequence_length=None,
                                    pad_to_max_tokens=False
                                    )

#Find the average number of tokens in the training tweets
round(sum([len(i.split()) for i in train_sentences])/len(train_sentences))

15

In [ ]:
#Now set up vectorization variables
max_vocab_length=10000
max_length=15

text_vectorizer = TextVectorization(max_tokens=max_vocab_length,
                                    output_mode='int',
                                    output_sequence_length=max_length,
                                    )

In [ ]:
#Fit the vectorizer on the training data
text_vectorizer.adapt(train_sentences)

In [ ]:
SAVE_DIR = 'model_logs'

In [ ]:
###Creating an embedding layer using Tensorflows embedding layer
from tensorflow.keras import layers

embedding = layers.Embedding(input_dim = max_vocab_length,
                             output_dim = 128,
                             input_length = max_length)
embedding

In [ ]:
#randomly choose a sample from train sentences and see
random_sentence = random.choice(train_sentences)
print (f'Original Sentence :\n {random_sentence}\
        \n\nEmbedded Version : \n')
embedding(text_vectorizer([random_sentence])) #because embedding layer takes integers as input and not words

Original Sentence :
 Arnhem Weather - &lt;p&gt;An unrelenting and dangerous heat wave will expand across the South Central United StatesÛ_ http://t.co/yhAqa5WXoK        

Embedded Version : 



<tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
array([[[ 0.01246893, -0.04003365, -0.01617442, ..., -0.01028466,
         -0.04201739,  0.03126859],
        [-0.00501598,  0.01560341, -0.02385846, ..., -0.04612421,
          0.02400445,  0.04464957],
        [-0.01291544, -0.01252741, -0.04572338, ...,  0.04718282,
         -0.01407555,  0.00111536],
        ...,
        [-0.01452354,  0.04780847,  0.01182637, ...,  0.04406799,
         -0.01557281, -0.00378921],
        [ 0.02225721,  0.0188919 , -0.02428427, ...,  0.01321859,
          0.02804179,  0.04404498],
        [-0.02219955,  0.04588038, -0.01859485, ...,  0.00600597,
         -0.04016919, -0.03021312]]], dtype=float32)>

###Lets build a RNN model now

###Model 2 - LSTM

In [ ]:
from tensorflow.keras import layers

inputs = layers.Input(shape=(1,), dtype='string')
x = text_vectorizer(inputs)
x = embedding(x)
print (x.shape)
x = layers.LSTM(64, return_sequences=True)(x)
print (x.shape)
x = layers.LSTM(64)(x)
print (x.shape)
x = layers.Dense(64, activation='relu')(x)
outputs = layers.Dense(1, activation='sigmoid')(x)
model_2 = tf.keras.Model(inputs, outputs, name='model_2_LSTM')

(None, 15, 128)
(None, 15, 64)
(None, 64)


In [ ]:
model_2.summary()

Model: "model_2_LSTM"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 lstm (LSTM)                 (None, 15, 64)            49408     
                                                                 
 lstm_1 (LSTM)               (None, 64)                33024     
                                                                 
 dense (Dense)               (None, 64)                4160      
                                                      

In [ ]:
model_2.compile(loss='binary_crossentropy',
                optimizer=tf.keras.optimizers.Adam(),
                metrics=['accuracy'])

In [ ]:
model_2_history = model_2.fit(train_sentences,
                              train_labels,
                              epochs=5,
                              validation_data = (val_sentences, val_labels),
                              callbacks=[create_tensorboard_callback(SAVE_DIR,
                                                                     'model_2_LSTM')])

Saving TensorBoard log files to: model_logs/model_2_LSTM/20220509-014605
Epoch 1/5
215/215 [==============================] - 10s 12ms/step - loss: 0.5148 - accuracy: 0.7396 - val_loss: 0.4578 - val_accuracy: 0.7861
Epoch 2/5
215/215 [==============================] - 2s 8ms/step - loss: 0.3191 - accuracy: 0.8711 - val_loss: 0.4675 - val_accuracy: 0.7756
Epoch 3/5
215/215 [==============================] - 2s 8ms/step - loss: 0.2208 - accuracy: 0.9197 - val_loss: 0.5698 - val_accuracy: 0.7638
Epoch 4/5
215/215 [==============================] - 2s 8ms/step - loss: 0.1621 - accuracy: 0.9409 - val_loss: 0.8214 - val_accuracy: 0.7585
Epoch 5/5
215/215 [==============================] - 2s 8ms/step - loss: 0.1185 - accuracy: 0.9539 - val_loss: 0.9412 - val_accuracy: 0.7572


In [ ]:
model_2_pred_probs = model_2.predict(val_sentences)
model_2_pred_probs[:10]

array([[0.00559152],
       [0.8082357 ],
       [0.99948883],
       [0.02843036],
       [0.00101362],
       [0.9998772 ],
       [0.39240554],
       [0.99990547],
       [0.9998423 ],
       [0.16618283]], dtype=float32)

In [ ]:
model_2_pred_probs = tf.squeeze(tf.round(model_2_pred_probs))
model_2_pred_probs[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 0., 1., 1., 0.], dtype=float32)>

In [ ]:
#Creating a function to calculate all the evaluation metrics

from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def calculate_results(y_true, y_pred):
  model_accuracy = accuracy_score(y_true, y_pred)*100
  model_precision, model_recall, f1_score, _ = precision_recall_fscore_support(y_true, y_pred, average='weighted')

  model_results = {"accuracy" : model_accuracy,
                   "precision" : model_precision,
                   "recall" : model_recall,
                   "f1_score" : f1_score}

  return model_results

In [ ]:
model_2_results = calculate_results(y_true=val_labels,
                                    y_pred=model_2_pred_probs)
model_2_results

{'accuracy': 75.7217847769029,
 'f1_score': 0.7548067945124027,
 'precision': 0.7588431452858484,
 'recall': 0.7572178477690289}

###Model 3 : GRU

In [ ]:
inputs = layers.Input(shape=(1,), dtype='string')
x = text_vectorizer(inputs)
x = embedding(x)
print (x.shape)
x = layers.GRU(64, return_sequences=True)(x)
print (x.shape)
x = layers.GRU(64)(x)
print (x.shape)
x = layers.Dense(64, activation='relu')(x)
outputs = layers.Dense(1, activation='sigmoid')(x)
model_3 = tf.keras.Model(inputs, outputs, name='model_3_GRU')

(None, 15, 128)
(None, 15, 64)
(None, 64)


In [ ]:
model_3.summary()

Model: "model_3_GRU"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 gru (GRU)                   (None, 15, 64)            37248     
                                                                 
 gru_1 (GRU)                 (None, 64)                24960     
                                                                 
 dense_2 (Dense)             (None, 64)                4160      
                                                       

In [ ]:
model_3.compile(loss='binary_crossentropy',
                optimizer=tf.keras.optimizers.Adam(),
                metrics=['accuracy'])

In [ ]:
model_3_history = model_3.fit(train_sentences,
                              train_labels,
                              epochs=5,
                              validation_data = (val_sentences, val_labels),
                              callbacks=[create_tensorboard_callback(SAVE_DIR,
                                                                     'model_3_GRU')])

Saving TensorBoard log files to: model_logs/model_3_GRU/20220509-014624
Epoch 1/5
215/215 [==============================] - 5s 11ms/step - loss: 0.2104 - accuracy: 0.9247 - val_loss: 0.6982 - val_accuracy: 0.7638
Epoch 2/5
215/215 [==============================] - 2s 8ms/step - loss: 0.1070 - accuracy: 0.9631 - val_loss: 0.9347 - val_accuracy: 0.7717
Epoch 3/5
215/215 [==============================] - 2s 8ms/step - loss: 0.0752 - accuracy: 0.9707 - val_loss: 0.9573 - val_accuracy: 0.7651
Epoch 4/5
215/215 [==============================] - 2s 7ms/step - loss: 0.0589 - accuracy: 0.9746 - val_loss: 1.3027 - val_accuracy: 0.7677
Epoch 5/5
215/215 [==============================] - 2s 8ms/step - loss: 0.0547 - accuracy: 0.9753 - val_loss: 1.1668 - val_accuracy: 0.7467


In [ ]:
model_3_pred_probs = model_3.predict(val_sentences)
model_3_pred_probs = tf.squeeze(tf.round(model_3_pred_probs))
model_3_results = calculate_results(y_true=val_labels,
                                    y_pred=model_3_pred_probs)
model_3_results

{'accuracy': 74.67191601049869,
 'f1_score': 0.7470840290954158,
 'precision': 0.7481917965802737,
 'recall': 0.7467191601049868}

###Model 4 : Bi-Directional LSTM

In [ ]:
inputs = layers.Input(shape=(1,), dtype='string')
x = text_vectorizer(inputs)
x = embedding(x)
print (x.shape)
x = layers.Bidirectional(layers.LSTM(64, return_sequences=True))(x)
print (x.shape)
x = layers.Bidirectional(layers.LSTM(64))(x)
print (x.shape)
x = layers.Dense(64, activation='relu')(x)
outputs = layers.Dense(1, activation='sigmoid')(x)
model_4 = tf.keras.Model(inputs, outputs, name='model_3_GRU')

(None, 15, 128)
(None, 15, 128)
(None, 128)


In [ ]:
model_4.summary()

Model: "model_3_GRU"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 bidirectional (Bidirectiona  (None, 15, 128)          98816     
 l)                                                              
                                                                 
 bidirectional_1 (Bidirectio  (None, 128)              98816     
 nal)                                                            
                                                       

In [ ]:
model_4.compile(loss='binary_crossentropy',
                optimizer=tf.keras.optimizers.Adam(),
                metrics=['accuracy'])

In [ ]:
model_4_history = model_4.fit(train_sentences,
                              train_labels,
                              epochs=5,
                              validation_data = (val_sentences, val_labels),
                              callbacks=[create_tensorboard_callback(SAVE_DIR,
                                                                     'model_4_BiDirectionalLSTM')])

Saving TensorBoard log files to: model_logs/model_4_BiDirectionalLSTM/20220509-014637
Epoch 1/5
215/215 [==============================] - 9s 20ms/step - loss: 0.1239 - accuracy: 0.9599 - val_loss: 1.3331 - val_accuracy: 0.7677
Epoch 2/5
215/215 [==============================] - 3s 12ms/step - loss: 0.0595 - accuracy: 0.9739 - val_loss: 1.6167 - val_accuracy: 0.7612
Epoch 3/5
215/215 [==============================] - 3s 12ms/step - loss: 0.0482 - accuracy: 0.9780 - val_loss: 1.3934 - val_accuracy: 0.7533
Epoch 4/5
215/215 [==============================] - 3s 12ms/step - loss: 0.0511 - accuracy: 0.9758 - val_loss: 1.3650 - val_accuracy: 0.7533
Epoch 5/5
215/215 [==============================] - 3s 12ms/step - loss: 0.0471 - accuracy: 0.9778 - val_loss: 1.5544 - val_accuracy: 0.7520


In [ ]:
model_4_pred_probs = model_4.predict(val_sentences)
model_4_pred_probs = tf.squeeze(tf.round(model_4_pred_probs))
model_4_results = calculate_results(y_true=val_labels,
                                    y_pred=model_4_pred_probs)
model_4_results

{'accuracy': 75.19685039370079,
 'f1_score': 0.7505489912204856,
 'precision': 0.7519989531509904,
 'recall': 0.7519685039370079}

###Model 5 : 1D-CNN for Text data

In [ ]:
#Testing how 1DCNN works on text data
embedding_test = embedding(text_vectorizer(['This is a test sentence']))
conv_1d = layers.Conv1D(filters =32,
                        kernel_size=5,
                        activation='relu',
                        padding='valid')
conv_1d_output = conv_1d(embedding_test)
max_pool = layers.GlobalMaxPool1D()
max_pool_output = max_pool(conv_1d_output)

embedding_test.shape, conv_1d_output.shape, max_pool_output.shape

(TensorShape([1, 15, 128]), TensorShape([1, 11, 32]), TensorShape([1, 32]))

In [ ]:
embedding_test

<tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
array([[[ 0.02898116, -0.01184049,  0.05657484, ...,  0.02664162,
         -0.01942774,  0.01999679],
        [-0.02407816,  0.04205197, -0.01866345, ..., -0.02455346,
         -0.02501391, -0.03456077],
        [-0.01177585, -0.00785562, -0.01970089, ...,  0.03952034,
         -0.01992994,  0.00255983],
        ...,
        [ 0.03388275,  0.01006775, -0.01794157, ...,  0.01198588,
         -0.00929688,  0.01212348],
        [ 0.03388275,  0.01006775, -0.01794157, ...,  0.01198588,
         -0.00929688,  0.01212348],
        [ 0.03388275,  0.01006775, -0.01794157, ...,  0.01198588,
         -0.00929688,  0.01212348]]], dtype=float32)>

In [ ]:
conv_1d_output

<tf.Tensor: shape=(1, 11, 32), dtype=float32, numpy=
array([[[0.00000000e+00, 0.00000000e+00, 2.22364552e-02, 0.00000000e+00,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
         0.00000000e+00, 1.36492159e-02, 8.47219676e-03, 0.00000000e+00,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
         8.15420691e-03, 6.97778016e-02, 6.97812531e-03, 1.81393418e-02,
         7.34966947e-03, 4.29819971e-02, 1.93657652e-02, 1.83961578e-02,
         9.28877213e-04, 1.23450924e-02, 1.07884444e-02, 3.56984213e-02,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 6.59300387e-02],
        [1.20106535e-02, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
         0.00000000e+00, 8.29476938e-02, 0.00000000e+00, 0.00000000e+00,
         1.23411203e-02, 4.20537256e-02, 4.58711525e-03, 0.00000000e+00,
         1.95399881e-03, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 4.92897853e-0

In [ ]:
max_pool_output

<tf.Tensor: shape=(1, 32), dtype=float32, numpy=
array([[0.08089843, 0.00365358, 0.02487618, 0.00877   , 0.04095142,
        0.08294769, 0.01853267, 0.        , 0.04285381, 0.04205373,
        0.06383035, 0.00344378, 0.01518647, 0.06190724, 0.0466614 ,
        0.05317757, 0.00815421, 0.0697778 , 0.01078461, 0.04928979,
        0.03571627, 0.042982  , 0.02749608, 0.08413976, 0.00092888,
        0.03393139, 0.04845641, 0.03569842, 0.        , 0.03403742,
        0.06480297, 0.06593004]], dtype=float32)>

###Now lets build a network based on 1DCNN for text data classification

In [ ]:
inputs = layers.Input(shape=(1,), dtype='string')
print (inputs.shape)
x = text_vectorizer(inputs)
x = embedding(x)
print (x.shape)
x = layers.Conv1D(filters=64, kernel_size=5, activation='relu', padding='valid')(x)
print (x.shape)
x = layers.GlobalAvgPool1D()(x)
print (x.shape)
x = layers.Dense(64, activation='relu')(x)
outputs = layers.Dense(1, activation='sigmoid')(x)
model_5 = tf.keras.Model(inputs, outputs, name='model_5_1DCNN')


model_5.compile(loss='binary_crossentropy',
                optimizer=tf.keras.optimizers.Adam(),
                metrics=['accuracy'])


model_5.summary()

(None, 1)
(None, 15, 128)
(None, 11, 64)
(None, 64)
Model: "model_5_1DCNN"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_6 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 conv1d_3 (Conv1D)           (None, 11, 64)            41024     
                                                                 
 global_average_pooling1d_2   (None, 64)               0         
 (GlobalAveragePooling1D)                                        
                                                                 
 

In [ ]:
model_5_history = model_5.fit(train_sentences,
                              train_labels,
                              epochs=5,
                              validation_data = (val_sentences, val_labels),
                              callbacks=[create_tensorboard_callback(SAVE_DIR,
                                                                     'model_5_1DCNN')])

Saving TensorBoard log files to: model_logs/model_5_1DCNN/20220509-020755
Epoch 1/5
215/215 [==============================] - 4s 8ms/step - loss: 0.1831 - accuracy: 0.9402 - val_loss: 0.8075 - val_accuracy: 0.7743
Epoch 2/5
215/215 [==============================] - 1s 5ms/step - loss: 0.0793 - accuracy: 0.9691 - val_loss: 1.0114 - val_accuracy: 0.7651
Epoch 3/5
215/215 [==============================] - 1s 5ms/step - loss: 0.0591 - accuracy: 0.9750 - val_loss: 1.1956 - val_accuracy: 0.7677
Epoch 4/5
215/215 [==============================] - 1s 5ms/step - loss: 0.0469 - accuracy: 0.9794 - val_loss: 1.3884 - val_accuracy: 0.7493
Epoch 5/5
215/215 [==============================] - 1s 5ms/step - loss: 0.0426 - accuracy: 0.9803 - val_loss: 1.5102 - val_accuracy: 0.7559


In [ ]:
model_5_pred_probs = model_5.predict(val_sentences)
model_5_pred_probs = tf.squeeze(tf.round(model_5_pred_probs))
model_5_results = calculate_results(y_true=val_labels,
                                    y_pred=model_5_pred_probs)
model_5_results

{'accuracy': 75.59055118110236,
 'f1_score': 0.75433369124314,
 'precision': 0.7562023016353725,
 'recall': 0.7559055118110236}